<a href="https://colab.research.google.com/github/std3rr/3D-Gaussian-Splatting-Real-Time-Lab/blob/main/gaussian_splatting_lab_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3D Gaussian splatting
https://repo-sam.inria.fr/fungraph/3d-gaussian-splatting/

This notebook is composed by Andreas Lewitzki.

I made this to experiment with processing video of coice, convert structure from motion and build a model for export to local computer for viewing.
As some packages and tools are compiled for CUDA support and from scratch it will take some time to bootstrap. (~35min)

Ofcourse you could take this and build a container image and deploy precompiled elsewhere.

In my lab I recorded 30s 4k video 30fps with my phone and uploaded directly to google drive.

I used a A100 instance for my initial lab.
It took about 4m30s for the colmap processing the 111 images (extracted @4fps) and generated roughly 20K points.

In this case the training did consume around up to almost 40G vram, but it would probably have worked running on a cheaper T4 instance if scaled down and cap after 7000 iterations. Training on A100 gave around 7.5 iterations/s while a T4 topped around 2.3 it/s. I opted for full resolution in this case, default is to scale down.

Training took 1h10m in this case.



#Install requirements and clone graphdecos gaussian-splatting project

Required dev environment for ceres-resolver and colmap

In [ ]:

!sudo apt-get install \
    git \
    cmake \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-regex-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libfreeimage-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libcgal-qt5-dev \
    libmetis-dev \
    libflann-dev \
    libatlas-base-dev \
    libsuitesparse-dev


 Ceres Solver 1 is an open source C++ library for modeling and solving large, complicated optimization problems. It can be used to solve Non-linear Least Squares problems with bounds constraints and general unconstrained optimization problems.

In [ ]:

!git clone https://ceres-solver.googlesource.com/ceres-solver
%cd ceres-solver
!git checkout $(git describe --tags) # Checkout the latest release
%mkdir build
%cd build
!cmake .. -DBUILD_TESTING=OFF -DBUILD_EXAMPLES=OFF -DCMAKE_CUDA_ARCHITECTURES=native
!make
!sudo make install

COLMAP is a general-purpose Structure-from-Motion (SfM) and Multi-View Stereo (MVS) pipeline with a graphical and command-line interface. It offers a wide range of features for reconstruction of ordered and unordered image collections.

In [ ]:
%cd ..
!git clone https://github.com/colmap/colmap
%cd colmap
!git checkout dev
%mkdir build
%cd build
!cmake .. -DCMAKE_CUDA_ARCHITECTURES=native
!make
!sudo make install
!CC=/usr/bin/gcc-6 CXX=/usr/bin/g++-6 cmake ..

clone graphdecos gaussian-splatting repo

In [ ]:
%cd /content
!git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting
!pip install -q plyfile

# borrow some precompiled python libs from camenduru
%cd /content/gaussian-splatting
!pip install -q https://huggingface.co/camenduru/gaussian-splatting/resolve/main/diff_gaussian_rasterization-0.0.0-cp310-cp310-linux_x86_64.whl
!pip install -q https://huggingface.co/camenduru/gaussian-splatting/resolve/main/simple_knn-0.0.0-cp310-cp310-linux_x86_64.whl



# Setup project
Mount google drive and grab your video file and splitt up depending on fps. So if you've recorded @ 30fps the output will be a png image representing every 15th frame.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
project_name = 'my3dgsproject1' # @param {type:"string"}
video_file = '/content/gdrive/MyDrive/test.MOV' # @param {type:"string"}
fps = 2 # @param {type:"slider", min:0, max:15, step:0.25}

!mkdir -p /content/{project_name}/input
!ffmpeg -i {video_file} -qscale:v 1 -qmin 1 -vf fps={fps} /content/{project_name}/input/%04d.jpg

# Process images and train model
This is essentially a wrapper over colmap feature extract, matcher, mapper and undistorter. <br>
After this we train model and upload to gdrive.

In [ ]:
%cd /content/gaussian-splatting
!python convert.py -s /content/{project_name}/

Train the model.
<b>-r 1</b> here means no resizing.

In [ ]:
!python train.py -s /content/{project_name} -r 1 --model_path=./output/{project_name}
!cp -R /content/{project_name} /content/gdrive/MyDrive/

# Now what?!
If you would like to just test the model localy, in my case windows, you may want to download:<br>
https://huggingface.co/camenduru/gaussian-splatting/resolve/main/GaussianViewTest.zip
<br><br>
If you have your google drive mounted in windows just unzip gaussianview then run something like:<br>
<i>GaussianViewTest\viewers\bin> .\SIBR_gaussianViewer_app.exe --model-path "gdrive\example\project_name"</i>

<br><br>
Bellow stuff is just for reference and optional.

In [ ]:
!python render.py -m /content/gaussian-splatting/output/{project_name}

In [ ]:
!ffmpeg -framerate 3 -i /content/gaussian-splatting/output/{project_name}/train/ours_30000/renders/%05d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -r 3 -pix_fmt yuv420p /renders.mp4
!ffmpeg -framerate 3 -i /content/gaussian-splatting/output/{project_name}/train/ours_30000/gt/%05d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -r 3 -pix_fmt yuv420p /gt.mp4 -y

In [ ]:
!wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
!unzip tandt_db.zip

!python train.py -s /content/gaussian-splatting/tandt/train

In [ ]:
!wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/GaussianViewTest.zip
!unzip GaussianViewTest.zip

